# Goal of laboratory work
The goal of this work is to implement a sequential and parallel k-means algorithm and compare the performance of the implemented algorithms.

# Task Definition
1. Describe the parallel implementations of the DBSCAN and k-means algorithms in terms of parallel design.
2. Select a data set among those proposed (or any other) and implement the k-means serial and parallel algorithm, Compare the performance of implemented algorithms.

# Brief Theory
The task of clustering is to divide the original data sample into disjoint groups, and the objects from one group should have a similarity, which is most often characterized by a function of the distance between objects. 



## DBSCAN
DBSCAN (Density-based spatial clustering of applications with noise) is a clustering algorithm that allows you to find clusters of arbitrary shape in a metric space. The basic idea of ​​the DBSCAN algorithm is to represent cluster objects as a group of points in a metric space, which are the vertices of a single connected graph.

Параллельные реализации алгоритма DBSCAN используют специальные пространственные структуры данных, такие как k-d tree, r-tree, vintage-point tree. Построение этих структур данных может быть распараллелено путем разделения множества объектов $X$ на части $X_i$, где $i=1,\dots,k$. 


The main feature of the parallel implementation of the DBSCAN algorithm is the master-slave architecture. The metric space is divided into areas and the calculations are distributed between the slave machines. Each of these areas is searched for clusters, and the need to merge with clusters from other areas is determined. Then the slave machines send the results of their calculations to the master machine, which merges the necessary clusters together and finishes the execution of the distributed algorithm.
The scheme of this algorithm is shown in the figure below.

![](images/dbscan.png)

## K-means
The k-means algorithm splits the set X into k sets $S_1, S_2, \dots, S_k$, so as to minimize the sum of squares of distances from each point of the cluster to its center. 
The work of the algorithm consists of  iteration, in each of which the distribution of d-dimensional vectors occurs over k clusters, as well as the recalculation of cluster centers in d-dimensional space. In the distribution step of d-dimensional vectors over k clusters, the distances between the vector and cluster centers are calculated independently, therefore they can be performed in parallel.

Then, dividing the entire calculation of distances into n streams, we obtain that in each stream only one operation will be performed to calculate the distance between the vectors. Moreover, the coordinates of the centers of all the clusters  are transferred to each computational flow.

Cluster mass centers are also recalculated independently of each other, these operations can be performed in separate threads.

# ALGORITHM (METHOD) of IMPLEMENTATION

In [10]:
%cat example2.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <sys/time.h>


#include <assert.h>
#include <float.h>
#include <math.h>
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

#include <omp.h>

#define KMEANS_NULL_CLUSTER -1

/*
* If the algorithm doesn't converge within this number of iterations,
* it will return with a failure error code.
*/
#define KMEANS_MAX_ITERATIONS 1000

#define kmeans_malloc(size) malloc(size)
#define kmeans_free(ptr) free(ptr)

typedef void * Pointer;

typedef enum {
	KMEANS_OK,
	KMEANS_EXCEEDED_MAX_ITERATIONS,
	KMEANS_ERROR
} kmeans_result;


typedef double (*kmeans_distance_method) (const Pointer a, const Pointer b);

typedef void (*kmeans_centroid_method) (const Pointer * objs, const int * clusters, size_t num_objs, int cluster, Pointer centroid);

typedef struct kmeans_config
{
	/* Function returns the "distance" between any pair of objects */
	kmeans_distance_method distance_method;

	/* Function returns the "centr

#  Result And Experiments

Clusterization is performed on the `birch3.txt` dataset from the http://cs.joensuu.fi/sipu/datasets/ website. We test how the number of threads affects the clusterization speed.

In [6]:
import subprocess
import os
from pathlib import Path

if Path('kmeans').exists():
    %cd kmeans/

def compile():
   
    _cmd = 'make -B example2'.split()
    # print(' '.join(_cmd))
    cmd = subprocess.run(_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  
    # if(cmd.stdout): print('cmd.stdout', cmd.stdout)
    if(cmd.stderr): print('cmd.stderr', cmd.stderr)
    
def run(env=None):
    cmd = subprocess.run('./example2', stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=env)
    

In [7]:


compile()

for th in [1, 2, 4, 8, 16]:
    env = os.environ.copy()
    env['OMP_NUM_THREADS'] = str(th)

    print(f"Executing kmeans with {th} threads", end="\n\t")
    %timeit run(env)
    print()



Executing kmeans with 1 threads
	526 ms ± 58.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Executing kmeans with 2 threads
	277 ms ± 3.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Executing kmeans with 4 threads
	169 ms ± 7.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Executing kmeans with 8 threads
	214 ms ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Executing kmeans with 16 threads
	215 ms ± 17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



#  Conclusion

In this paper, k-means sequential and parallel clustering algorithms were implemented and tested on a real word dataset.
The skill of using clustering methods and the OpenMP library was gained.
